 ## 2Vn4UzR5W1_Assignement1_NLP

In [55]:
import numpy as np
import pandas as pd
import re
import string

# Upload the different data

### 1- The mapping data 
This help to understand how the target value are encoded

In [100]:
mapping = pd.read_csv("sentiment/mapping.txt",header= None, delimiter = "\t")
mapping.head()

,0,1
0,0,negative
1,1,neutral
2,2,positive


### 2- The different target for the train, test and validation

In [11]:
Y_train = pd.read_csv("sentiment/train_labels.txt", header = None)
Y_train.head()

,0
0,2
1,1
2,1
3,1
4,2


In [12]:
Y_test = pd.read_csv("sentiment/test_labels.txt", header= None)
Y_test.head()

,0
0,1
1,1
2,1
3,2
4,0


In [13]:
Y_val = pd.read_csv("sentiment/val_labels.txt", header= None)
Y_val.head()

,0
0,1
1,2
2,0
3,1
4,1


### 3- The Different tweet for train, validation and test

In [78]:
# Specify the path to your text file
file_path = 'sentiment/train_text.txt'

# Read the text file into a list of strings (each line is an element in the list)
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Create a DataFrame with each line as a separate observation
data = {'Tweet': lines}
X_train = pd.DataFrame(data)

# Display the DataFrame
X_train.head()

,Tweet
0,"""QT @user In the original draft of the 7th boo..."
1,"""Ben Smith / Smith (concussion) remains out of..."
2,Sorry bout the stream last night I crashed out...
3,Chase Headley's RBI double in the 8th inning o...
4,@user Alciato: Bee will invest 150 million in ...


In [15]:
# Specify the path to your text file
file_path = 'sentiment/test_text.txt'

# Read the text file into a list of strings (each line is an element in the list)
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Create a DataFrame with each line as a separate observation
data = {'Tweet': lines}
X_test = pd.DataFrame(data)

# Display the DataFrame
X_test.head()

,Tweet
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."


In [16]:
# Specify the path to your text file
file_path = 'sentiment/val_text.txt'

# Read the text file into a list of strings (each line is an element in the list)
with open(file_path, 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Create a DataFrame with each line as a separate observation
data = {'Tweet': lines}
X_val = pd.DataFrame(data)

# Display the DataFrame
X_val.head()

,Tweet
0,Dark Souls 3 April Launch Date Confirmed With ...
1,"""National hot dog day, national tequila day, t..."
2,When girls become bandwagon fans of the Packer...
3,@user I may or may not have searched it up on ...
4,Here's your starting TUESDAY MORNING Line up a...


# A- Preprocing of the Data
In the tweet data, the emoticon and emojis are really important to judge the impression of the twiter.So first thing to do as preprocessing is to convert emoticon and emojis to word. For each data set, we are goinf to applied the same preprocessing process. 

## 1 - Convert the emoticons to word

Here we save all the Emoticons and their description in the file emoticons_dict.txt

In [17]:
#file_path = 'emoticons_dict.txt'

# with open(file_path, 'w', encoding='utf-8') as txt_file:
#     for emoticon, description in EMOTICONS.items():
#         txt_file.write(f"{emoticon}: {description}\n")

# print(f"Emoticons and descriptions have been written to {file_path}")


In [18]:
def load_emoticons_dict(file_path):
    
    """ create a function that load our emoticon 
    file It take as input the path of the file and return the dictionnary """
    
    emoticons_dict = {}
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        for line in txt_file:
            emoticon, description = line.strip().split(': ')
            emoticons_dict[emoticon] = description
    return emoticons_dict

def Replace_emoticons(text, emoticons_dict):
    
    """
    return the description of emoticon in the text input
    
    """
    for emoticon, description in emoticons_dict.items():
        text = text.replace(emoticon, f'[{description}]')
    return text

def convert_emoticons(text,emoticons_dict ):
    
    for emot in emoticons_dict:
        text = re.sub(u'('+emot+')', " ".join(emoticons_dict[emot].replace(",","").split()), text)
    return text

# Load emoticons dictionary from the text file
loaded_emoticons_dict = load_emoticons_dict('emoticons_dict.txt')

# Example usage
text_with_emoticons = "I'm so happy! :D This is amazing! :-3"
cleaned_text = Replace_emoticons(text_with_emoticons,loaded_emoticons_dict)
cleaned_text2 = convert_emoticons(text_with_emoticons, loaded_emoticons_dict )
print("Original text:", text_with_emoticons)
print("Cleaned text:", cleaned_text)
print("Cleaned text2:", cleaned_text2)


Original text: I'm so happy! :D This is amazing! :-3
Cleaned text: I'm so happy! [Laughing, big grin or laugh with glasses] This is amazing! [Happy face smiley]
Cleaned text2: I'm so happy! Laughing big grin or laugh with glasses This is amazing! Happy face smiley


In [19]:
convert_emoticons("I am sad :( ",loaded_emoticons_dict)

'I am sad Frown sad andry or pouting '

In [20]:
Replace_emoticons("I am sad :( ", loaded_emoticons_dict)

'I am sad :( '

In [21]:
Replace_emoticons("I am happy :-\)\)", loaded_emoticons_dict)

'I am happy [Happy face smiley]\\)'

In [22]:
convert_emoticons("I am happy :-\)\)", loaded_emoticons_dict)

'I am happy :-\\Confusion\\Confusion'

We can notice that there are some emoticon that the first function convert well but don't convert well another.The second function replace don't reconnaisse another emoticon. we will apply the two function first the replace and then the convert.

For each dataset we go through each line and applied ours functions.

### a- Apply replace_emoticons function it in the validation text

In [23]:
# Create an empty dataset to store the result 
X_val_re_emoti = pd.DataFrame(index=range(X_val.shape[0]), columns=["Tweet"])
for i in range(X_val.shape[0]):
    X_val_re_emoti["Tweet"][i] = Replace_emoticons(X_val["Tweet"][i], loaded_emoticons_dict)
    X_val_re_emoti["Tweet"][i] = convert_emoticons(X_val_re_emoti["Tweet"][i], loaded_emoticons_dict)
X_val_re_emoti.head()

,Tweet
0,Dark Souls 3 April Launch Date Confirmed With ...
1,"""National hot dog day, national tequila day, t..."
2,When girls become bandwagon fans of the Packer...
3,@user I may or may not have searched it up on ...
4,Here's your starting TUESDAY MORNING Line up a...


### b- Apply replace_emoticons function it in the text text

In [24]:
# Create an empty dataset to store the result 
X_test_re_emoti = pd.DataFrame(index=range(X_test.shape[0]), columns=["Tweet"])
for i in range(X_test.shape[0]):
    X_test_re_emoti["Tweet"][i] = Replace_emoticons(X_test["Tweet"][i], loaded_emoticons_dict)
    X_test_re_emoti["Tweet"][i] = convert_emoticons(X_test_re_emoti["Tweet"][i], loaded_emoticons_dict)
X_test_re_emoti.head()

,Tweet
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."


### c- Apply replace_emoticons function it in the train text¶

In [79]:
# Create an empty dataset to store the result 
X_train_re_emoti = pd.DataFrame(index=range(X_train.shape[0]), columns=["Tweet"])
for i in range(X_test.shape[0]):
    X_train_re_emoti["Tweet"][i] = Replace_emoticons(X_train["Tweet"][i], loaded_emoticons_dict)
    X_train_re_emoti["Tweet"][i] = convert_emoticons(X_train_re_emoti["Tweet"][i], loaded_emoticons_dict)
X_train_re_emoti.head()

,Tweet
0,"""QT @user In the original draft of the 7th boo..."
1,"""Ben Smith / Smith (concussionConfusion remain..."
2,Sorry bout the stream last night I crashed out...
3,Chase Headley's RBI double in the 8th inning o...
4,@user Alciato: Bee will invest 150 million in ...


Verifie if the function work well with one observation.


In [26]:
X_train["Tweet"][12]

'Super excited for homecoming Saturday with Monroe! :D my new nephew is here but I can\\u2019t see him :( \n'

In [27]:
X_train_re_emoti["Tweet"][12]

'Super excited for homecoming Saturday with Monroe! [Laughing, big grin or laugh with glasses] my new nephew is here but I can\\u2019t see him Frown sad andry or pouting \n'

## 2 - Convert the emojis to word
We did the same thing as we the emoticon

In [28]:
#file_path = 'emojis_dict.txt'

# with open(file_path, 'w', encoding='utf-8') as txt_file:
#     for emoji, description in UNICODE_EMO.items():
#         txt_file.write(f"{emoji}: {description}\n")

# print(f"Emoticons and descriptions have been written to {file_path}")

In [29]:
def load_emoji_dict(file_path):
    
    """ create a function that load our emoji
    file It take as input the path of the file and return the dictionnary """
    
    emojis_dict = {}
    with open(file_path, 'r', encoding='utf-8') as txt_file:
        for line in txt_file:
            emoji, description = line.strip().split(': ')
            emojis_dict[emoji] = description
    return emojis_dict

def replace_emojis(text,emojis_dict ):
    
    """
    return the description of emoji in the text input
    
    """
    for emoji, description in emojis_dict.items():
        text = text.replace(emoji, f'[{description}]')
    return text

def convert_emojis(text,emojis_dict ):
    for emot in emojis_dict:
        text = re.sub(r'('+emot+')', "_".join(emojis_dict[emot].replace(",","").replace(":","").split()), text)
    return text

# Load emoticons dictionary from the text file
loaded_emojis_dict = load_emoji_dict('emojis_dict.txt')

# Example usage
text_with_emoji = "game is on 🔥"
cleaned_text = replace_emojis(text_with_emoji, loaded_emojis_dict)

print("Original text:", text_with_emoji)
print("Cleaned text:", cleaned_text)

Original text: game is on 🔥
Cleaned text: game is on [:fire:]


In the result data where we replace emoticon by
their description in word we apply the function replace_emojis which replace the emojis to their descriptions. 

### a- Apply replace_emojis function it in the validation text


In [30]:
# Create an empty dataset to store the result 
X_val_re_emoji = pd.DataFrame(index=range(X_val_re_emoti.shape[0]), columns=["Tweet"])
for i in range(X_val.shape[0]):
    X_val_re_emoji["Tweet"][i] = replace_emojis(X_val_re_emoti["Tweet"][i], loaded_emojis_dict)
X_val_re_emoji.head()

,Tweet
0,Dark Souls 3 April Launch Date Confirmed With ...
1,"""National hot dog day, national tequila day, t..."
2,When girls become bandwagon fans of the Packer...
3,@user I may or may not have searched it up on ...
4,Here's your starting TUESDAY MORNING Line up a...


### b- Apply replace_emojis function it in the test text

In [31]:
# Create an empty dataset to store the result 
X_test_re_emoji = pd.DataFrame(index=range(X_test_re_emoti.shape[0]), columns=["Tweet"])
for i in range(X_test.shape[0]):
    X_test_re_emoji["Tweet"][i] = replace_emojis(X_test_re_emoti["Tweet"][i], loaded_emojis_dict)
X_test_re_emoji.head()

,Tweet
0,@user @user what do these '1/2 naked pics' hav...
1,OH: “I had a blue penis while I was this” [pla...
2,"@user @user That's coming, but I think the vic..."
3,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast..."


We verifie in two observations in the dataset

In [32]:
X_test_re_emoti["Tweet"][6]

'How many more days until opening day? 😩 \n'

In [33]:
X_test_re_emoji["Tweet"][6]

'How many more days until opening day? [:weary_face:] \n'

In [34]:
X_test_re_emoti["Tweet"][269]

'Spirit of my silence i can hear you. But im afraid to be near you. #ThisIsUs 😍😍😍 \n'

In [35]:
X_test_re_emoji["Tweet"][269]

'Spirit of my silence i can hear you. But im afraid to be near you. #ThisIsUs [:smiling_face_with_heart-eyes:][:smiling_face_with_heart-eyes:][:smiling_face_with_heart-eyes:] \n'

In [86]:
X_train_re_emoti.isna().sum()

Tweet    0
dtype: int64

### c- Apply replace_emojis function it in the train text

In [101]:
# Create an empty dataset to store the result 
X_train_re_emoji = pd.DataFrame(index=range(X_train_re_emoti.shape[0]), columns=["Tweet"])

# Check for missing values in the "Tweet" column
#X_train_re_emoti["Tweet"] = X_train_re_emoti["Tweet"].fillna("")

for i in range(X_train.shape[0]):
    X_train_re_emoji["Tweet"]= replace_emojis(X_train_re_emoti["Tweet"][i], loaded_emojis_dict)
X_train_re_emoji.head()

,Tweet
0,
1,
2,
3,
4,


It seem like there isn't emojis in the train dataset.

## 3- Remove Punctuation

We remove the punctuation in all our dataset the previous cleaning result.and create a new column call tweet_no_puntuation 

In [57]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

In [92]:
PUNCT_TO_REMOVE

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

### a- Apply this function in validation data

In [97]:
X_Train = X_train_re_emoti

# Check for missing values in the "Tweet" column
X_Train["Tweet"] = X_Train["Tweet"].fillna("")

X_Train["text_wo_punct"] = X_Train["Tweet"].apply(lambda Tweet: remove_punctuation(Tweet))
X_Train.head()

,Tweet,text_wo_punct
0,"""QT @user In the original draft of the 7th boo...",QT user In the original draft of the 7th book ...
1,"""Ben Smith / Smith (concussionConfusion remain...",Ben Smith Smith concussionConfusion remains o...
2,Sorry bout the stream last night I crashed out...,Sorry bout the stream last night I crashed out...
3,Chase Headley's RBI double in the 8th inning o...,Chase Headleys RBI double in the 8th inning of...
4,@user Alciato: Bee will invest 150 million in ...,user Alciato Bee will invest 150 million in Ja...


### b- Apply this function in test data

In [89]:
X_Test =  X_test_re_emoji
X_Test["text_wo_punct"] = X_Test["Tweet"].apply(lambda Tweet: remove_punctuation(Tweet))
X_Test.head()

,Tweet,text_wo_punct
0,@user @user what do these '1/2 naked pics' hav...,user user what do these 12 naked pics have to ...
1,OH: “I had a blue penis while I was this” [pla...,OH “I had a blue penis while I was this” playi...
2,"@user @user That's coming, but I think the vic...",user user Thats coming but I think the victims...
3,I think I may be finally in with the in crowd ...,I think I may be finally in with the in crowd ...
4,"@user Wow,first Hugo Chavez and now Fidel Cast...",user Wowfirst Hugo Chavez and now Fidel Castro...


### c- Apply this function in train data

In [90]:
X_Val =  X_val_re_emoji
X_Val["text_wo_punct"] = X_Val["Tweet"].apply(lambda Tweet: remove_punctuation(Tweet))
X_Val.head()

,Tweet,text_wo_punct
0,Dark Souls 3 April Launch Date Confirmed With ...,Dark Souls 3 April Launch Date Confirmed With ...
1,"""National hot dog day, national tequila day, t...",National hot dog day national tequila day then...
2,When girls become bandwagon fans of the Packer...,When girls become bandwagon fans of the Packer...
3,@user I may or may not have searched it up on ...,user I may or may not have searched it up on g...
4,Here's your starting TUESDAY MORNING Line up a...,Heres your starting TUESDAY MORNING Line up at...


## 4- Find Non Alphanumeric

In [50]:
def find_non_apl_words(text):
    tokens = text.strip().split()
    return [t for t in tokens if not re.match(r'[^\W\d]*$', 
                                              t.translate(str.maketrans('', '', r'\.\"\,\?\!\@\'()'))
                                             )]

In [98]:
# Create an empty dataset to store the result 
X_train_non_apha_num = pd.DataFrame(index=range(X_Train.shape[0]), columns=["Tweet"])
X_train_non_apha_num = X_Train['Tweet'].apply(lambda x: find_non_apl_words(x))
X_val_non_apha_num

0        [7th, #HappyBirthdayRemusLupin"]
1                         [/, #NHL, #SJ"]
2                                      []
3                               [8th, 33]
4             [Alciato:, 150, 200, 2017"]
                       ...               
45610                                  []
45611                                  []
45612                                  []
45613                                  []
45614                                  []
Name: Tweet, Length: 45615, dtype: object

### b- Apply this function in test data

In [103]:
# Create an empty dataset to store the result 
X_test_non_apha_num = pd.DataFrame(index=range(X_Test.shape[0]), columns=["Tweet"])
X_test_non_apha_num = X_Test['Tweet'].apply(lambda x: find_non_apl_words(x))
X_test_non_apha_num

0                                                   ['1/2]
1                          [OH:, “I, this”, [playing, VR]]
2                                                       []
3                        [#mannequinchallenge, #grads2014]
4                                                       []
                               ...                        
12279                            [Editorial:, FBI’s, ‘had]
12280                                           [#vanessa]
12281    [#latestnews, 4, #newmexico, #politics, +, #na...
12282                                           [#caveman]
12283                                                   []
Name: Tweet, Length: 12284, dtype: object

In [ ]:
X_Test['Tweet'][0]

### c- Apply this function in train data

In [104]:
# Create an empty dataset to store the result 
X_val_non_apha_num = pd.DataFrame(index=range(X_Val.shape[0]), columns=["Tweet"])
X_val_non_apha_num = X_Val['Tweet'].apply(lambda x: find_non_apl_words(x))
X_val_non_apha_num

0                                 [3, Trailer:]
1                                            []
2                                         [1st]
3                                            []
4       [9[Happy, smiley]0, 10[Happy, smiley]0]
                         ...                   
1995                                         []
1996                                         []
1997               [#007, #SPECTRE, #JamesBond]
1998                [5, [Skeptical,, hesitant]]
1999                                     [16th]
Name: Tweet, Length: 2000, dtype: object

We notice that all our three datasets has non Alphanumerique caracter.

## 3 - Spelling Correction

In [106]:
from spellchecker import SpellChecker

spell = SpellChecker()
def correct_spellings(text):
    corrected_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            corrected_text.append(spell.correction(word))
        else:
            corrected_text.append(word)
    return " ".join(corrected_text)

text = "speling correctin"
correct_spellings(text)

'spelling correcting'

In [117]:
def correct_spellings(text):
    corrected_text = []
    for word in text.split():
        # Correct the spelling if it's in the dictionary, otherwise keep the original word
        corrected_word = spell.correction(word)
        corrected_text.append(corrected_word) if corrected_word else None

    # Filter out None values and join the corrected text
    return " ".join(filter(None, corrected_text))

In [118]:
text = "speling correctin"
correct_spellings(text)

'spelling correcting'

We apply this function on the result for the previous cleaned

### a- In validation set

In [ ]:
# Create an empty dataset to store the result 
X_val_spell = pd.DataFrame(index=range(X_Val.shape[0]), columns=["text_wo_punct"])
for i in range(X_val.shape[0]):
    X_val_spell["text_wo_punct"][i] = correct_spellings(X_Val["text_wo_punct"][i])
X_val_spell.head()

In [110]:
X_Val["text_wo_punct"][270]

'Mar do you know that Caitlyn Jenner still has a ball sack  my 7yr old cousin \n'

In [114]:
correct_spellings(X_Val["text_wo_punct"][27])

'every time I hear alright by Kendrick I think its j Coles Black Friday'

In [45]:
X_val_re_emoji["Tweet"][12]

'We just received more tickets for Blue Rodeo at The KEE to Bala Saturday May 19th and Sunday May 20th. Tickets... \n'

## 4 - Chat Words Conversion

In [113]:
chat_words_map_dict = {}
chat_words_list = []
for line in chat_words_str.split("\n"):
    if line != "":
        cw = line.split("=")[0]
        cw_expanded = line.split("=")[1]
        chat_words_list.append(cw)
        chat_words_map_dict[cw] = cw_expanded
chat_words_list = set(chat_words_list)

def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

chat_words_conversion("one minute BRB")

'one minute Be Right Back'

In [112]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
KISS=Keep It Simple, Stupid
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [ ]:
apply this in all our Dataset 

In [1]:
!pip install pyspellchecker

Just to understand how the sentimental analysis work 

In [ ]:
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize

# Download the punkt tokenizer if not already downloaded
nltk.download('punkt')
nltk.download('vader_lexicon')

# Sample data
text_data = [
    "Nice! I have the bottom two!",
    "Happy Friday everyone! Coffee is on...",
    "Having to read Charles Dickens for homework...",
    "I am going to see Aesop Rock and Rob Sonic on Fri Aug 3rd...",
    "Crazy that you use Kendrick. I'm listening to Kendrick Lamar...",
    "South Korea May Be China Telecom's Next Foreign Conquest..."
    # ... add more texts from your data
]

# Initialize the Sentiment Intensity Analyzer
sid = SentimentIntensityAnalyzer()

# Analyze sentiment for each text
for text in text_data:
    # Tokenize the text
    words = word_tokenize(text)

    # Calculate sentiment scores
    sentiment_scores = sid.polarity_scores(text)

    # Print the results
    print(f"Text: {text}")
    print(f"Sentiment Scores: {sentiment_scores}")
    print("-----")


In [ ]:
sentiment_scores = sid.polarity_scores("so happy Laughing big grin laugh with glasses is amazing Heart")
sentiment_scores 

In [ ]:
sid.polarity_scores("so happy This is amazing ")

In [ ]:
#!pip install nltk 